In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import pygad

module_path = os.path.abspath(os.path.join('../')) # or the path
sys.path.append(module_path)

import itertools
import pandas as pd
import numpy as np

In [50]:
from engine import Bool
from bnn import ProdTron, SumTron, ProdLayer, SumLayer

# can we learn an xor gate?
def optim_xor(x,w,y, model):
    h = [ xi^wi for xi, wi in zip(x,w)]
    yh = np.prod(h) # 
    yhd = [yh.data]
    
    # run one iteration of GD
    loss = y^yh # xor between observed and predicted
    loss.backward()

    flips = 0

    # for this specific case, gradient update takes this form
    for wi in w:
        if wi.grad == wi.data:
            flips += 1
            wi.data = -wi.data
    
    # re-eval the gate
    h = [ xi^wi for xi, wi in zip(x,w)]
    yh = np.prod(h) #
    yhd.append(yh.data)

    mistake = 0

    if loss.data<0:
        print('model is CORRECT')    
        if flips>0:
            print('#',flips,'BUT got updated ---')
            print('model after: ',[wi.data for wi in w])
            mistake += 1
        else:
            print('and NOT updated +++')
    else:
        print('model is WRONG.') 
        if flips>0:
            print('#',flips,'and got UPDATED +++')
            print('model after: ',[wi.data for wi in w])
        else:
            mistake +=1
            print('but NOT UPDATED ---')
    
    print('pred before: ',yhd[0],'\npred  after: ',yhd[1])
    return w,yhd,flips,mistake


import itertools
T = [-1,1]
mistakes = 0
for element in itertools.product(T, repeat=5):
    print('\n***\n')
    y = Bool(element[0])
    x = [Bool(element[1]),Bool(element[2])]
    w = [Bool(element[3]),Bool(element[4])]
    print('input: ',x[0].data, x[1].data)
    print('y: ',y.data)
    print('model: ',w[0].data, w[1].data)
    w,yhd,flips,mistake = optim_xor(x,w,y)
    mistakes += mistake
print('total mistakes: ', mistakes, 'out of', np.power(2,5))



***

input:  -1 -1
y:  -1
model:  -1 -1
model is CORRECT
and NOT updated +++
pred before:  -1 
pred  after:  -1

***

input:  -1 -1
y:  -1
model:  -1 1
model is CORRECT
and NOT updated +++
pred before:  -1 
pred  after:  -1

***

input:  -1 -1
y:  -1
model:  1 -1
model is CORRECT
and NOT updated +++
pred before:  -1 
pred  after:  -1

***

input:  -1 -1
y:  -1
model:  1 1
model is WRONG.
# 2 and got UPDATED +++
model after:  [-1, -1]
pred before:  1 
pred  after:  -1

***

input:  -1 1
y:  -1
model:  -1 -1
model is CORRECT
and NOT updated +++
pred before:  -1 
pred  after:  -1

***

input:  -1 1
y:  -1
model:  -1 1
model is CORRECT
and NOT updated +++
pred before:  -1 
pred  after:  -1

***

input:  -1 1
y:  -1
model:  1 -1
model is WRONG.
# 2 and got UPDATED +++
model after:  [-1, 1]
pred before:  1 
pred  after:  -1

***

input:  -1 1
y:  -1
model:  1 1
model is CORRECT
and NOT updated +++
pred before:  -1 
pred  after:  -1

***

input:  1 -1
y:  -1
model:  -1 -1
model is CORRECT
an

The term we want to model is $x_1 \neg x_2$ or $\neg x_1 x_2$. What we notice is, gradients are not defined (or ignored) when needed (model is wrong) in some specific cases. 

That is, when $x_i=w_i$ and $x_1=x_2$, the model is $AND(\neg x_1 \neg x_2) = F$. Local gradient of $AND(.,.)$ is 0 in this case.

This seems like a problem. When no other signal is present (for particular combination of inputs, model weights, and label to model), model updates can not happen via backprop.

Do we need to modify the definitions of the gradient operator and/or the definition of the gates.

For example, based on the definition according to BOLD, 

$AND(x,0) =0 \forall x$. But this can be modified as follows.

In fact, when $x = T \equiv x > 0$, we could have affirmatively defined the gradient to be $AND(x,0) = T \text{ when } x=T$. But when $x < 0$, it is undecidable and information from the other input is needed to determine the quadrant. May be we can throw a coin to decide. At least, in expectation, we will be right.

So, the modified gates with extended Boolean space , with fuzzy o/p, can be defined as:

**Fuzzy AND gate**

| $x_1$ | $x_2$ | $y_{AND}$ | 
|-----|-----|-----|
T| T | T | 
T| F | F |
F| T | F | 
F| F | F |
T| 0 | T |
F| 0 | $\text{Bernoulli}(0.5)$ |
0| T | T |
0| F | $\text{Bernoulli}(0.5)$ |
0| 0 | $\text{Bernoulli}(0.5)$ |

As a result, $AND(x_1,x_2) \in   \{T,F\} \forall x \in \{T,0,F\}$, i.e.,  even if any of the inputs are 0, by definition, AND gate is not 0.

We need to extend this to $XNOR$ gate, as the derivative involves $XNOR$ gate.

**Fuzzy XNOR gate**

| $x_1$ | $x_2$ | $y_{XOR}$ | 
|-----|-----|-----|
T| T | T | 
T| F | F |
F| T | F | 
F| F | T |
T/F| 0 | $\text{Bernoulli}(0.5)$ |
0| T/F | $\text{Bernoulli}(0.5)$ |
0| 0 | $\text{Bernoulli}(0.5)$ |

#### Derivative of AND

Recall:
1. $\delta(a \to b) \equiv True$ if $b > a$, $\equiv 0$ if $b = a$, and $\equiv False$ if $b < a$.
2. $f'(x) \equiv \text{xnor}(\delta(x \to \neg x), \delta f(x \to \neg x))$.

The Truth Table for $f(x) = f_a(x) = AND(x,a)$ is:

| $a$ | $x$ | $\neg x$ | $\delta x$ | $f(x)$ |$f(\neg x)$ |  $\delta f$ | $f'$ 
|-----|-----|-----|-----| -----| -----|-----|-----|
T| T | F | F | T | F | F | T | 
T| F | T | T | F | T | T | T | 
F| T | F | F | F | F | 0 | $\text{Bernoulli}(0.5)$ | |
F| F | T | T | F | F | 0 | $\text{Bernoulli}(0.5)$ | |

Therefore, $f'_{a}(x) \in \mathcal{B} \forall a,x \in \mathcal{B}$. Effectively, when the output can not be decided, we toss a fair coin decide. Once we ensure that logic gates are in $\mathcal{B}$, we do not encounter the problem we saw with the definitions according to BOLD.

But before we go this route, we have another question. To learn the expression, will contrastive examples be sufficient. Let us explore this next.





In [54]:
import itertools

# we want to realize x1*x2' term. The truth table for this term is
T = [[-1,-1,-1],[-1,1,-1],[1,-1,1],[1,1,-1]]

mistakes = 0
w = [Bool(-1),Bool(-1)]

# for some rows in the truth table, this model is wrong
# by looping through other data, will we be able to eventually update the model?

for element in T:
    print('\n***\n')
    x = [Bool(element[0]),Bool(element[1])]
    y = Bool(element[2])    
    print('input: ',x[0].data, x[1].data)
    print('y: ',y.data)
    print('model: ',w[0].data, w[1].data)
    w,yhd,flips,mistake = optim_xor(x,w,y)
    mistakes += mistake
# final model
print(w)

# see if it is correct for all inputs
for element in T:
    print('\n***\n')
    x = [Bool(element[0]),Bool(element[1])]
    yh = np.prod([ xi^wi for xi, wi in zip(x,w)])
    if yh.data == element[2]:
        print('correct')
    else:
        print('wrong')


***

input:  -1 -1
y:  -1
model:  -1 -1
model is CORRECT
and NOT updated +++
pred before:  -1 
pred  after:  -1

***

input:  -1 1
y:  -1
model:  -1 -1
model is CORRECT
and NOT updated +++
pred before:  -1 
pred  after:  -1

***

input:  1 -1
y:  1
model:  -1 -1
model is WRONG.
# 1 and got UPDATED +++
model after:  [-1, 1]
pred before:  -1 
pred  after:  1

***

input:  1 1
y:  -1
model:  -1 1
model is CORRECT
and NOT updated +++
pred before:  -1 
pred  after:  -1
[data:-1, grad:1, data:1, grad:-2]

***

correct

***

correct

***

correct

***

correct


In [ ]:
# we should verify this for all different initializations of the weights

# we want to realize x1*x2' term. The truth table for this term is
T = [[-1,-1,-1],[-1,1,-1],[1,-1,1],[1,1,-1]]

mistakes = 0
w = [Bool(-1),Bool(-1)]

# for some rows in the truth table, this model is wrong
# by looping through other data, will we be able to eventually update the model?

for element in T:
    print('\n***\n')
    x = [Bool(element[0]),Bool(element[1])]
    y = Bool(element[2])    
    print('input: ',x[0].data, x[1].data)
    print('y: ',y.data)
    print('model: ',w[0].data, w[1].data)
    w,yhd,flips,mistake = optim_xor(x,w,y)
    mistakes += mistake
# final model
print(w)

# see if it is correct for all inputs
for element in T:
    print('\n***\n')
    x = [Bool(element[0]),Bool(element[1])]
    yh = np.prod([ xi^wi for xi, wi in zip(x,w)])
    if yh.data == element[2]:
        print('correct')
    else:
        print('wrong')